# Secret Santa

### Objectives
1. Read `participants.csv` where eachand assign a secret santa assignment for each participant, given the following constraints:
    - each participant cannot be assigned themself nor significant other
    - each participant can only have one assignement and be assigned once
2. Text each participant with instructions and their secret santa assignment

In [3]:
import pandas as pd
import numpy as np
from twilio.rest import Client

In [8]:
participants = pd.read_csv('participants.csv', index_col='id')
participants.columns

Index([u'couple_id', u'name', u'phone_number'], dtype='object')

### 1. make assignments

In [9]:
make_assignments = True
while make_assignments:
    id_l = participants.index.tolist()
    participants['assigned_id'] = None
    participants['assigned_couple_id'] = None
    participants['assigned_name'] = None

    for index, row in participants.iterrows():
        # escape before hitting infite loop scenario
        if (len(id_l) == 1) & (id_l[0] == index):
            break
        
        needs_assignment = True

        while needs_assignment:
            rand_id = np.random.randint(low=0,high=len(id_l))
            participants.loc[index, 'assigned_id'] = id_l[rand_id]
            participants.loc[index, 'assigned_couple_id'] = participants.couple_id.loc[id_l[rand_id]]      
            participants.loc[index, 'assigned_name'] = participants.name.loc[id_l[rand_id]]

            if participants.loc[index, 'couple_id'] != participants.loc[index, 'assigned_couple_id']:
                id_l.remove(id_l[rand_id])
                needs_assignment = False
        
        # escape before hitting infite loop scenario
        if (len(id_l) == 2):
            remaining_couples_l = participants.couple_id.ix[id_l].tolist()
            if (remaining_couples_l[0] == remaining_couples_l[1]):
                break
    
    if len(participants[participants.assigned_id.isnull()]) == 0:
        make_assignments = False
        
        

participants.columns

/Users/jojensen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Index([u'couple_id', u'name', u'phone_number', u'assigned_id',
       u'assigned_couple_id', u'assigned_name'],
      dtype='object')

In [10]:
# save to csv
participants.to_csv('participants_with_assignments.csv')

### 2. text instructions and assignments

In [2]:
with open('twilio_api.txt') as f:
    twilio_creds = f.read().split('\n')
    
twilio_client = Client(twilio_creds[0], twilio_creds[1])

In [11]:
participants.head()

,couple_id,name,phone_number,assigned_id,assigned_couple_id,assigned_name
id,,,,,,
1,1,Angela,15629004561,11,7,Bruce
2,1,Josh,15629004561,4,2,Laurie
3,2,Roy,15629004561,6,3,Shirley (aka Mem)
4,2,Laurie,15629004561,7,4,Dan
5,3,Steve (aka FT),15629004561,8,4,Mia


In [15]:
for index, row in participants.iterrows():
    twilio_client.messages.create(
        to = row['phone_number'],
        from_ = twilio_creds[2],
        body = '''
Hello %s, welcome to Jensen Christmas Scoopganza 2017! 

ScoopBot here to spread Christmas cheer, and today that means facilitating Secret Santa.

The rules of Secret Santa are easy:
1. I assign you someone else from the group. Keep it secret, keep it safe (until X-Mas day).
2. You buy $15 - $25 worth of shit that will blow their minds.
3. On Christmas day, blow their minds with your generousity and thoughtfulness. 

Hold onto your butt, because your secret santa assignment is:
%s

If you don't know %s very well, I'd suggest asking Ang or Josh. They might be able to point you in the right direction.

Remember don't let %s know you are their secret santa until Christmas day!

SCOOP!
''' % (row['name'], row['assigned_name'], row['assigned_name'], row['assigned_name'])
    )
    
    print 'Sent %s of %s' % (index, len(participants))
    
    break

Sent 1 of 11


In [ ]:
#test
twilio_client.messages.create(
    to = '1509712219'
    from_ = twilio_creds[2]
    body = 'TEST'
)